# Phase de pretraitement des données textuelles

In [15]:
import pandas as pd
from clean import clean_claimKG

import nltk
nltk.download('all')
!pip3 install inflect
import inflect 
import contractions as c
import re
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/massy/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/massy/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/massy/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/massy/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/massy/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/massy/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]   

In [2]:
file_name = "../data/claimKG.csv"

# Lecture du fichier
kg_origin = pd.read_csv(file_name)
kg= kg_origin.copy()
kg = clean_claimKG(kg)

In [34]:
claims = kg['claimReview_claimReviewed']
claims_text = []
for k,v in claims.sample(10).items():
    claims_text.append(v)

for claim in claims_text:
    print("->",claim)

-> Medline is donating money to breast cancer-related charities in conjunction with a “Pink Glove” promotion.
-> Did Planned Parenthood Perform More Abortions Last Year Than Any Year Since 2011-2012?
-> Trump’s tax plan is ""completely focused on the wealthy and the powerful - not the middle class.""
-> Washing instruction tags on Tom Bihn bags carry a hidden message in French disparaging the president.
-> HIV-positive basketball star Earvin ""Magic"" Johnson donated blood to patients with leukemia.
-> ""We saw the prison population expand and vacancy rates for correctional officers go from 2.9% to 15.2%"" over the last eight years.
-> California has seen a ""56 percent decline in the gun murder rate"" since the 1990s.
-> Georgia’s illegal-immigration legislation ""simply follows existing federal law.""
-> ""Kesha Rogers is not a Democrat.""
-> Emma González was photographed attacking a truck belonging to a ""Second Amendment supporter"" during a March for Our Lives event in March 2018

## Tokenization

Découpage de l'assertion en Token (en mots).

In [77]:
def tokenize(text):
    return nltk.word_tokenize(text)

tokenize(claims_text[8])

['``', '``', 'Kesha', 'Rogers', 'is', 'not', 'a', 'Democrat', '.', "''", "''"]

## Mise en miniscule

La mise en miniscule peut-être util dans certain cas.

In [43]:
def lowercase(text):
    return text.lower()

lowercase(claims_text[2])

'trump’s tax plan is ""completely focused on the wealthy and the powerful - not the middle class.""'

## numbers to words

Transformer les nombres en mots.

In [70]:
def number_to_words(text):
    return inflect.engine().number_to_words(text)

print(number_to_words("15.2"))

print(claims_text[5])

def number2words(text):
    tokens = tokenize(text)
    for i,m in enumerate(tokens):
        try:
            tokens[i] = number_to_words(float(m))
        except ValueError:
            continue
    return ' '.join(tokens)

print(number2words(claims_text[5]))

fifteen point two
""We saw the prison population expand and vacancy rates for correctional officers go from 2.9% to 15.2%"" over the last eight years.
`` `` We saw the prison population expand and vacancy rates for correctional officers go from two point nine % to fifteen point two % '' '' over the last eight years .


## Traitement des contraction et ponctuation

La suppression des ponctuations peut avoir des conséquences sur le qualité du modèle, par exemple dans la détection des opinions. Il est préferable de traiter d'abord les contracions dans les phrases avant de supprimer les ponctuations.

In [73]:
import contractions as c
def contractions(text):
    return c.fix(text)

print(contractions("couldn't"))

def ponctuations(text):
    return re.sub(r'[^\w\s]', ' ', text)

print(ponctuations(claims_text[5]))

could not
couldn t
  We saw the prison population expand and vacancy rates for correctional officers go from 2 9  to 15 2    over the last eight years 


## Stopwords

Supprimer les mots les plus fréquent de la langue. Dans notre cas 

NB: Dans les stopwords fournit par défault par NLTK contient les formes de négation.

In [84]:
print(stopwords.words('english'))
stopwords.words('french')

def remove_stopwords(text_tokenized):
    stop_words = set(stopwords.words('english'))
  
    return [w for w in text_tokenized if not w in stop_words]

print(claims_text[8])
claim = ponctuations(claims_text[8])
claim = tokenize(claim)
claim = remove_stopwords(claim)
print(' '.join(claim))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Stematisation

## Lemmatisation

## Pos-tagging

two point nine


AttributeError: 'function' object has no attribute 'fix'